In [ ]:

#DataSet is available in below link
#http://stats.espncricinfo.com/ci/engine/records/team/series_results.html?id=159;type=trophy


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time

In [27]:
def LaunchURL(URL):
    # create a new Firefox session
    driver1 = webdriver.Firefox(executable_path=r'C:\Users\shubh\Jupyter\BPL\geckodriver.exe')
    driver1.implicitly_wait(10)
    driver1.get(URL)
    return driver1

In [28]:
def SeriesPage(driver):
    return driver.find_elements_by_class_name('data-link')

In [29]:
def AllMatchDetails(driver):
    return driver.find_elements_by_class_name('potMatchLink')

In [30]:
def RedirectToCommetry(driver, Inning):
    NewURL = driver.current_url
    NewURL = NewURL.replace('/scorecard/','/commentary/')+'?innings=' +  str(Inning)
    driver.get(NewURL)
    return driver


In [31]:
def FindDivElement(driver):
    Frame = driver.find_element_by_id('pane-main')
    Frame2 = Frame.find_element_by_id('main-container')
    return Frame2.find_elements_by_class_name('main-content')[0]

In [48]:
#launch url
url = "http://stats.espncricinfo.com/ci/engine/records/team/series_results.html?id=159;type=trophy"
driverMain = LaunchURL(url)

In [ ]:
# get All Session data 
data = []
SeriesLinks = SeriesPage(driverMain)
for EachSeries in SeriesLinks:
    #click on Each Series
    print ('-------------------------------------' + str(EachSeries.get_attribute('innertext')))
    
    #EachSeries.click()
    SessionURL = EachSeries.get_attribute('href')
    driver = LaunchURL(SessionURL)
    MatchLinks = AllMatchDetails(driver)
    for match in MatchLinks:
        # Click on Each Match
        #print (match.find_element_by_tag_name('a').get_attribute('href'))
        #match.click()
        #time.sleep(0.5)
        #matchURL = driver.current_url
        try:
            driverTemp = LaunchURL(match.find_element_by_tag_name('a').get_attribute('href'))
            #driver.execute_script('window.history.go(-1)')
            # Re Direct to Commetry Tab
            for Inning in range(1,3):
                driverTemp = RedirectToCommetry(driverTemp, Inning)
                #Scroll Down
                #ScrollDown(driver)

                # Find Main Div Element
                div = FindDivElement(driverTemp)
                blnTaskComplete =False
                SCROLL_PAUSE_TIME = 0.5
                for i in range (0,15):
                    html = driverTemp.find_element_by_tag_name('html')
                    html.send_keys(Keys.END)
                    time.sleep(SCROLL_PAUSE_TIME)
                # get End Over Details
                # div.find_elements_by_tag_name('h4')
                # get Each Ball Details

                div = FindDivElement(driverTemp)
                for eachball in div.find_elements_by_class_name('commentary-item'):
                    record = eachball.get_property('innerText')
                    if ' to ' in record:
                        if str(record.splitlines()[0]).isnumeric :
                            data.append (record.splitlines())
                        else:
                            data.append (record.splitlines()[2:])
            driverTemp.close()
        except:
            pass
    df = pd.DataFrame(data=data)
    df.to_csv('BPL.csv')                                

-------------------------------------None


In [ ]:
#https://www.espncricinfo.com/series/8653/scorecard/551045/barisal-burners-vs-sylhet-royals-1st-match-bangladesh-premier-league-2011-12
#https://www.espncricinfo.com/series/8653/commentary/551045/barisal-burners-vs-sylhet-royals-1st-match-bangladesh-premier-league-2011-12

In [ ]:
div.find_elements_by_tag_name('h4')[5].get_property('innerText')

In [ ]:
div.find_elements_by_class_name('commentary-item')[6].get_property('innerText')